# TEMPLATE JAWABAN TUGAS LANGUAGE MODELING

### Import Library

In [1]:
import nltk
import xlrd
import operator
import random
import math
nltk.download('punkt')
nltk.download('wordnet')

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\HP\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\HP\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

### Import Dataset (Training dan Testing)

In [2]:
book = xlrd.open_workbook('dataTrainArticle.xlsx')
book2 = xlrd.open_workbook('dataTestArticle.xlsx')
sheet = book.sheet_by_name('Sheet1')
sheet2 = book2.sheet_by_name('Sheet1')
dataTrain = [[sheet.cell_value(r, c) for c in range(sheet.ncols)] for r in range(sheet.nrows)]
dataTest = [[sheet2.cell_value(r, c) for c in range(sheet2.ncols)] for r in range(sheet2.nrows)]
#add dummy-ish data for OOV (word that doesn't exist in data train), the OOV will be converted to _____
#just 4 basic word classes, noun, verbs, adjectives, adverb
#without this p(oov) will be 0
dataTrain.append(['_____ has _____ the _____']) 
#<noun> has <verb> the <noun>
dataTrain.append(['_____  is one _____ car'])
#<noun> is one <adjective> car
dataTrain.append(['_____  is _____ _____ a new car'])
#<noun> is <adverb> <noun> a new car
print('done')

done


### Bangun Model Unigram

In [3]:
def buildUnigramModel(trainingSentences):
    #init var
    wordCount = {} #format wordCount[w(i)] = prob(w(i)), ex wordCount['the'] = 0.454
    count = 0
    wordCount['<s>'] = 0
    wordCount['</s>'] = 0
    #count word
    for sentence in trainingSentences:
        tokens = nltk.word_tokenize(sentence[0]) #break article to word
        for token in tokens: #for the actual word
            if token == '.':
                wordCount['<s>'] += 1
                wordCount['</s>'] += 1
                count += 2
            else:
                tokenLower = token.lower() #to generalize the letter
                if tokenLower in wordCount:
                    wordCount[tokenLower] += 1
                else:
                    wordCount[tokenLower] = 1
                count += 1
    unigramModel = wordCount
    #count prob
    for word in unigramModel:
        unigramModel[word] = unigramModel[word] / count
    return unigramModel
unigramModel = buildUnigramModel(dataTrain)
print('done')

done


### Bangun Model Bigram

In [13]:
def buildBigramModel(trainingSentences):
    #init
    wordPairCount = {} #format wordPairCount[w(i - 1)][w(i)] = prob(w(i) | w(i - 1)), ex. wordPairCount['nissan']['gt-r'] = 0.38
    wordCount = {} #to count the divider for each word
    wordCount['<s>'] = 0
    wordCount['</s>'] = 0
    wordPairCount['<s>'] = {} #there's no wordPairCount['</s>'] because there's nothing after </s>
    #count word
    for sentence in trainingSentences:
        tokens = nltk.word_tokenize(sentence[0]) #break sentence to word
        tokenBefore = '<s>' #for the very first word in the article
        for token in tokens:
            if token == '.':
                if '</s>' in wordPairCount[tokenBefore]:
                    wordPairCount[tokenBefore]['</s>'] += 1
                else:
                    wordPairCount[tokenBefore]['</s>'] = 1
                tokenBefore = '<s>'
                wordCount['<s>'] += 1
                wordCount['</s>'] += 1
            else:
                tokenLower = token.lower() #to generalize the letter
                if tokenLower not in wordPairCount: #to save any word to the model as the word before
                    wordPairCount[tokenLower] = {}
                if tokenLower in wordPairCount[tokenBefore]: #to count the word based on word before
                    wordPairCount[tokenBefore][tokenLower] += 1
                else:
                    wordPairCount[tokenBefore][tokenLower] = 1
                tokenBefore = tokenLower #save the word before
                if tokenLower in wordCount: #count each word
                    wordCount[tokenLower] += 1
                else:
                    wordCount[tokenLower] = 1
    bigramModel = {}  #init too
    #filling the rest of the model count with 0, so that the "index error" error won't happen
    #and also count the probability
    for wordBefore in wordPairCount.keys():
        bigramModel[wordBefore] = {}
        for theWord in wordCount.keys():
            if theWord != '<s>': #because <s> is the first, no word before
                if theWord not in wordPairCount[wordBefore]:
                    wordPairCount[wordBefore][theWord] = 0
                bigramModel[wordBefore][theWord] = wordPairCount[wordBefore][theWord] / wordCount[wordBefore]
    return bigramModel
bigramModel = buildBigramModel(dataTrain)
print('done')

done


### Bangun Fungsi Kata Berikutnya Dengan Probability Tertinggi

In [38]:
#Diberikan sebuah kata, cari kata berikutnya yang memiliki probability tertinggi berdasarkan model bigram
#proportional random from top n word
#param model, a word, n (top n word)
def nextBestWord(bigramModel, currentWord, topn):
    #init
    top = []
    holder = {}
    currentWordLow = currentWord.lower() #to make the case same with the model
    if currentWordLow not in bigramModel: #for OOV word (that is not in the training/model)
        currentWordLow = "_____"
    for i in range(topn):
        maxs = max(bigramModel[currentWordLow].items(), key=operator.itemgetter(1))[0] #get max prob
        holder[maxs] = bigramModel[currentWordLow][maxs] #save the max prob
        top.append(maxs) #save the max word
        bigramModel[currentWordLow].pop(maxs) #pop the max word, so that the next loop get the second max
    for i in range(topn): #put the popped back to the model
        bigramModel[currentWordLow].update( {top[i] : holder[top[i]]} )
    #proportional random
    maks = sum(holder.values())
    pick = random.uniform(0, maks)
    current = 0
    for key, value in holder.items():
        current += value
        if current > pick:
            return key
#Lakukan percobaan dengan memberikan sebuah kata, lalu print kata berikutnya
print(nextBestWord(bigramModel,"porsche",3))

cayenne


### Hitung probabilitas bigram menggunakan metode interpolasi (interpolation)

In [44]:
#count the p(w(i) | w(i - 1)) using the simple interpolation
#params unigram model, bigram model, lambda unigram, lambda bigram, w(i - 1), w(i)
def countInterpolationBigram(unigramModel, bigramModel, lambdauni, lambdabi, wordBefore, theWord):
    if bigramModel[wordBefore][theWord] == 0: #because math.log2(0) crash the program
        return (lambdabi * 0) + (lambdauni * math.log2(unigramModel[theWord]))
    else:
        return (lambdabi * math.log2(bigramModel[wordBefore][theWord])) + (lambdauni * math.log2(unigramModel[theWord]))

### Hitung Perplexity Unigram dan Bigram terhadap Data Testing

In [59]:
def countPerplexity(unigramModel, bigramModel, testSentences):
    perplexity = {} #init
    #unigram
    count1 = 0 #the M
    total1 = 0 #the l
    for sentence in testSentences:
        tokens = nltk.word_tokenize(sentence[0]) #break sentence to words
        for word in tokens:
            if word == '.':
                '''we need to include the begin- and end-sentence markers <s> and
                </s> in the probability computation. We also need to include the end-of-sentence
                marker </s> (but not the beginning-of-sentence marker <s>) in the total count of
                word tokens N. (from jurafsky's book)'''
                total1 += math.log2(unigramModel['</s>'])
                total1 += math.log2(unigramModel['<s>'])
                count1 += 1
            else:
                wordLow = word.lower() #to make the case same with the model
                if wordLow not in unigramModel: #for OOV word (that is not in the training/model)
                    wordLow = "_____"
                total1 += math.log2(unigramModel[wordLow])
                count1 += 1 #for the word
    total1 = total1 / count1 #total * 1 / M
    total1 = total1 * -1 #negative total
    perplexity1 = 2 ** total1 #2^-l
    perplexity.update({'unigram' : perplexity1})
    #bigram with simple interpolation (one of the smoothing method)
    lambdauni = 0.5 #lambda unigram model
    lambdabi = 0.5 #lambda bigram model
    count2 = 0 #the M
    total2 = 0 #the l
    for sentence in testSentences:
        tokens = nltk.word_tokenize(sentence[0]) #break sentence to words
        wordBefore = '<s>'
        for word in tokens:
            count2 += 1
            if word == '.':
                total2 += countInterpolationBigram(unigramModel, bigramModel, lambdauni, lambdabi, wordBefore, '</s>') #end of sentence
                wordBefore = '<s>'
            else:
                wordLow = word.lower() #to make the case same with the model
                if wordLow not in bigramModel or wordLow not in unigramModel:
                    wordLow = "_____" #for OOV word (that is not in the training/model)
                total2 += countInterpolationBigram(unigramModel, bigramModel, lambdauni, lambdabi, wordBefore, wordLow)
                wordBefore = wordLow #save the w(i - 1)
    total2 = total2 / count2 #total * 1 / M
    total2 = total2 * -1 #negative total
    perplexity2 = 2 ** total2 #2^-l
    perplexity.update({'bigram' : perplexity2})
    return perplexity
print(countPerplexity(unigramModel, bigramModel, dataTest))

{'unigram': 746.8009860634719, 'bigram': 56.707027552706975}


### Bangun Fungsi Generate Kalimat

In [63]:
#params, the model, number of word, and top-n for the generator
def generateSentence(model, length, top):
    sentence = '' #init
    wordBefore = '<s>' #very first word
    for i in range(length):
        while(True): #to avoid generating _____ (the OOV dummy model)
            nextWord = nextBestWord(model,wordBefore,top)
            if nextWord != '_____':
                break
        if nextWord == '<s>':
            sentence += ' '
        elif nextWord == "</s>":
            sentence += '.'
        else:
            #for better spacing
            if nextWord != "'" and nextWord != '’' and wordBefore != '‘' and wordBefore != '’' and wordBefore != '“' and wordBefore != '(' and nextWord != '”' and nextWord != '"' and nextWord != ',' and nextWord != '(' and nextWord != ')':
                sentence += ' '
            sentence += nextWord
        wordBefore = nextWord
    return sentence
generateSentence(bigramModel, 50, 25)

' the original. has been mooted. a whole shebang will be built into the most obvious lack of cars like any surprises. a ‘siblings. is all that could drop a car, but now. the car called the company has just look grey body roll'